In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from IPython.core.display import display, HTML

In [2]:
source = './data-input.xlsx'
source_filter = 'Eth'

In [3]:
def get_definitions(data, form_name, location):
    forms = []
    for index, col in enumerate(list(data)):
        datatype = data[col].dtypes
        formtype = "text"
        options = None
        if datatype == int:
            formtype = "number"
        if datatype == np.float64:
            formtype = "decimal"
        if datatype == object:
            test = data[col].dropna()
            options = list(test.unique())
            if len(options) > 8:
                options = None
                formtype = "text"
            else:
                formtype = "option"
        if col in location:
            options = None
            formtype = "cascade"
        if formtype == "option":
            for opt in options:
                forms.append({"id": index,"question": col, "type": formtype, "options": opt})
        else:
            forms.append({"id": index, "question": col, "type": formtype, "options": " - "})
    display(HTML(f"<h1>{form_name}</h1>"))
    display(pd.DataFrame(forms).groupby(['id','question','type','options']).first())
    display(HTML(f"<hr/>"))

In [4]:
all_sheets = load_workbook(source, read_only=True).sheetnames
sheets = list(filter(lambda x: 'Eth' in x, all_sheets))

for sheet in sheets:
    data = pd.read_excel(source, sheet)
    get_definitions(data, sheet, ["Woreda","Kebele"])

Empty DataFrame
Columns: []
Index: [(0, Woreda, cascade,  - ), (1, Kebele, cascade,  - ), (2, Village, text,  - ), (3, Name of Respondent, text,  - ), (4, Household Size, number,  - ), (5, Main Source of Drinking Water, text,  - ), (6, Water Service Level, option, Basic), (6, Water Service Level, option, Limited), (6, Water Service Level, option, Surface Water), (6, Water Service Level, option, Unimproved), (7, Specific Location of Water Collection, option, Elsewhere), (7, Specific Location of Water Collection, option, In own dwelling), (7, Specific Location of Water Collection, option, In own yard / plot), (8, Time to Collect Water, option, Less than 30 minutes), (8, Time to Collect Water, option, More than 30 minutes), (8, Time to Collect Water, option, less than 30 minutes), (9, Times in Last Month When Drinking Water Quantity Was Insufficient, option, Don't know), (9, Times in Last Month When Drinking Water Quantity Was Insufficient, option, No, always sufficient), (9, Times in Last Month When Drinking Water Quantity Was Insufficient, option, Yes, at least once), (10, Sanitation Service Level, option, Basic), (10, Sanitation Service Level, option, Limited), (10, Sanitation Service Level, option, Safely Managed), (10, Sanitation Service Level, option, Unimproved), (11, Type of Toilet Facility, option, Bucket), (11, Type of Toilet Facility, option, Pit latrine with slab), (11, Type of Toilet Facility, option, Pit latrine without slab/open pit), (11, Type of Toilet Facility, option, There is no latrine), (11, Type of Toilet Facility, option, Toilet that flush/pour to flush piped sewer system), (11, Type of Toilet Facility, option, Toilet that flush/pour to pit latrine), (11, Type of Toilet Facility, option, Twinpit offset latrine), (11, Type of Toilet Facility, option, Ventilated Improved Pit latrine (VIP)), (12, Facility is Shared with Others Outside of the Household, option, No), (12, Facility is Shared with Others Outside of the Household, option, Yes), (13, Location of Sanitation Facility, option, Elsewhere), (13, Location of Sanitation Facility, option, In own dwelling), (13, Location of Sanitation Facility, option, In own plot/yard), (14, Emptying of On-site Sanitation Facilities, option, Don't know), (14, Emptying of On-site Sanitation Facilities, option, No, never emptied), (14, Emptying of On-site Sanitation Facilities, option, Yes, but don't know when), (15, Hygiene Service Level, option, Basic), (15, Hygiene Service Level, option, Limited), (15, Hygiene Service Level, option, No Facility), (16, Type of Handwashing Facility Used Most Often, option, Fixed facility observed in dwelling), (16, Type of Handwashing Facility Used Most Often, option, Fixed facility observed in plot/yard), (16, Type of Handwashing Facility Used Most Often, option, Mobile object observed (bucket, jug, kettle)), (16, Type of Handwashing Facility Used Most Often, option, No handwashing place available in dwelling/plot/yard), (16, Type of Handwashing Facility Used Most Often, option, No permission to see), (17, Water Available at Handwashing Facility, option, Water is available), (17, Water Available at Handwashing Facility, option, Water is not available), (18, Soap is Available, option, Ash/mud/sand is present), (18, Soap is Available, option, No, not present), (18, Soap is Available, option, Yes, soap or detergent are present)]

Empty DataFrame
Columns: []
Index: [(0, Woreda, cascade,  - ), (1, Kebele, cascade,  - ), (2, Name_of_Health_facility, text,  - ), (3, Latitude, decimal,  - ), (4, Longitude, decimal,  - ), (5, Type_of_healthy_facility, option, Health Post), (5, Type_of_healthy_facility, option, Other), (6, Description of Water Supply in Health Facilities, option, No water supply in premises), (6, Description of Water Supply in Health Facilities, option, Pipeline Connections), (6, Description of Water Supply in Health Facilities, option, Protected Wells), (6, Description of Water Supply in Health Facilities, option, Rain Water Harvesting), (7, Water, option, Basic), (7, Water, option, Limited), (7, Water, option, No Service), (8, Latrine_Description, option, No Latrine), (8, Latrine_Description, option, Simple pit latrine), (8, Latrine_Description, option, VIP), (9, Sanitation, option, Limited), (9, Sanitation, option, No Service), (9, Sanitation, option, Usable), (10, Physically_Separate_Latrines_for_Male_Female_Patients, option, No), (11, Facilitated_Access_for_Disabled, option, No), (12, Latrine_condition, option, Clean, used), (12, Latrine_condition, option, Dirty but used), (12, Latrine_condition, option, Filthy ,unused), (13, 992194004|Is_there_hand_washing_facility_with_in_5m_of_the_toilet, option, No Facility), (14, Hygiene, option, No Service), (15, Is_hand_washing_facility_operating_i.e_with_water, option, Yes), (16, Is_Soap_available_at_each_hand_washing_stand, option, No), (17, Is_there_a_toilet_for_staff, option, No)]

Empty DataFrame
Columns: []
Index: [(0, Woreda, cascade,  - ), (1, Kebele, cascade,  - ), (2, School Name, text,  - ), (3, Latitude, decimal,  - ), (4, Longitude, decimal,  - ), (5, School Type, option, High School 9-10), (5, School Type, option, Primary(1-4)), (5, School Type, option, Primary(1-8)), (6, Male Pupils, number,  - ), (7, Female Pupils, decimal,  - ), (8, Reason for Inventory, option, New Scheme), (8, Reason for Inventory, option, Repair), (9, Water Supply Source, option, No Water Scheme), (9, Water Supply Source, option, No Water Supply In Premises), (9, Water Supply Source, option, Pipeline Connection), (9, Water Supply Source, option, Shallow Well fitted with Pump), (10, Water, option, Basic), (10, Water, option, Limited), (10, Water, option, No Service), (11, Year Commisioned, decimal,  - ), (12, Functionality_Status_of_Water_Supply, option, Functional), (12, Functionality_Status_of_Water_Supply, option, Non Functional - Mechanical), (13, Yield, decimal,  - ), (14, Type of Latrine, option, No Latrine), (14, Type of Latrine, option, Simple Pit Latrine), (15, Sanitation, option, Limited), (15, Sanitation, option, No Service), (16, latrine_For_Boys_and_Girls_Separate, option, No), (16, latrine_For_Boys_and_Girls_Separate, option, Yes), (17, Number of Boy Pupils, number,  - ), (18, Number of Girl Pupils, decimal,  - ), (19, Are there Latrines for Disabled?, option, No), (19, Are there Latrines for Disabled?, option, Yes), (20, Latrine_Condition, option, Clean, Used), (20, Latrine_Condition, option, Dirty But Used), (20, Latrine_Condition, option, No Latrine), (21, Hygiene SErvice Level, option, No Service), (22, Presence_of_HandWashingFacility, option, No), (22, Presence_of_HandWashingFacility, option, Yes), (23, hand_washing_in_use, option, No), (24, availability_of_soap, option, No), (25, Urinal_present_girls, option, No), (25, Urinal_present_girls, option, Yes), (26, Urinal_present_boys, option, No), (26, Urinal_present_boys, option, Yes), (27, Separate_latrine_for_tolitet_Staff, option, No), (27, Separate_latrine_for_tolitet_Staff, option, Yes), (28, Unnamed: 28, decimal,  - ), (29, Unnamed: 29, decimal,  - ), (30, Unnamed: 30, decimal,  - ), (31, Unnamed: 31, decimal,  - ), (32, Unnamed: 32, decimal,  - ), (33, Unnamed: 33, decimal,  - ), (34, Unnamed: 34, decimal,  - ), (35, Unnamed: 35, decimal,  - ), (36, Unnamed: 36, decimal,  - ), (37, Unnamed: 37, decimal,  - ), (38, Unnamed: 38, decimal,  - ), (39, Unnamed: 39, decimal,  - )]

Empty DataFrame
Columns: []
Index: [(0, Woreda, cascade,  - ), (1, Kebele, cascade,  - ), (2, Village, text,  - ), (3, Site Name, text,  - ), (4, Latitude, decimal,  - ), (5, Longitude, decimal,  - ), (6, Water Source Type_1, option, Deep Well with Distribution), (6, Water Source Type_1, option, Hand dug Well fitted with pump or windlass), (6, Water Source Type_1, option, Protected Spring), (6, Water Source Type_1, option, Shallow Well fitted with hand pump), (7, Water Source Type, option, Deep Wel), (7, Water Source Type, option, Hand dug Well), (7, Water Source Type, option, Hand dug well), (7, Water Source Type, option, Protected Spring), (7, Water Source Type, option, Shallow Well), (7, Water Source Type, option, Shallow well), (7, Water Source Type, option, Spring), (8, Functionality Status, option, Functional), (8, Functionality Status, option, No Data), (8, Functionality Status, option, No data), (8, Functionality Status, option, Non Functional), (8, Functionality Status, option, Not functional), (9, Souce of Energy, option, Diesel), (9, Souce of Energy, option, Electricity and Grid), (9, Souce of Energy, option, Gravity), (9, Souce of Energy, option, Manual operation), (9, Souce of Energy, option, Other), (9, Souce of Energy, option, Solar), (10, Estimated Number of Users, text,  - ), (11, Hand Pump Type, option, Afridev), (11, Hand Pump Type, option, Indian Mark II), (11, Hand Pump Type, option, Submersible), (12, Organisation that installed the Water Pump, option, CDA), (12, Organisation that installed the Water Pump, option, Government), (12, Organisation that installed the Water Pump, option, Kale Hiwot), (13, Year Commisioned, decimal,  - ), (14, Reason for Inventory, option, Extension), (14, Reason for Inventory, option, Inspection), (14, Reason for Inventory, option, National WASH inventory), (14, Reason for Inventory, option, New Scheme), (14, Reason for Inventory, option, Repair), (15, Functional Taps In The Scheme, decimal,  - ), (16, Number of Non FunctionalTaps In The Scheme, decimal,  - ), (17, Depth, decimal,  - ), (18, Yield, decimal,  - )]

,,,
id,question,type,options
0,Woreda,cascade,-
1,Kebele,cascade,-
2,Village,text,-
3,Latitude,decimal,-
4,Longitude,decimal,-
5,No. of HHs,decimal,-
6,Initial Number Latrines,decimal,-
7,Final Number of Latrines,decimal,-
8,Date Triggered,text,-
